### Páginas 521-534: Aprende ML con Scikit-Learn, Keras y Tensorflow (Versión 2023)

- Clasificación y localización:

    La localización de un objeto en una imagén puede expresarse como un modelo de regresión de 4 parametros, para poder predecir un cuadro delimitador alrededor del objeto. Lo normal es predecir: Coordenadas horizontar y vertical del centro del objeto además de la altura y la anchura. De esta manera, deberemos tener dos capas de salida de la red neuronal: 1 para la clasificación del objeto y otra para la regresión de dichos 4 parametros del modelo(una capa densa final con 4 neuronas y f.activacion por defecto. Un ejemplo de red neuronal para visión por computador pàra clasificación y localización de objetos:




In [ ]:
base_model = tf.keras.applications.xception.Xception(weights="imagenet",
                                                     include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
class_output = tf.keras.layers.Dense(n_classes, activation="softmax")(avg)
loc_output = tf.keras.layers.Dense(4)(avg)
model = tf.keras.Model(inputs=base_model.input,
                       outputs=[class_output, loc_output])
model.compile(loss=["sparse_categorical_crossentropy", "mse"],
              loss_weights=[0.8, 0.2],  # depends on what you care most about
              optimizer=optimizer, metrics=["accuracy"])


    Problema: a menudo es complicado y costoso tener el conjunto de los datos con los cuadros delimitadores alrededor de los objetos

    Inciso: Los cuadros delimitadores dberían normarlizarse de manera que las coordenadas horizontales y verticales,así como la altyura y la anchura vayan de 0 a 1.También es habitual predecir la raíz cuadrada de la anchura y de la altura, de manera que no se penalizen tanto los errores de los pixeles para un cuadro delimitador grande.

    Muchas veces el ECM como función de pèrdida para predecir estos parametros funciona bien pero la métrica más común para este tipo de problemas de la "interseccion sobre union":calculada dividiendo el area de solapamiento por el cuadro delimitador predicho y el cuadro delimitador objetivo entre el area de la union de ambos cuadros.

- Detección de objetos: ¿Pero y si en una imágen tenemos que clasificar y localizar varios objetos?El enfoque habitual durante el pasado ha sido el de coger una RNC entrenada ypara clasificar y localizar un solo objeto más o menos centrado en la imagen, y posteriormente deslizarla a través de la imágen para hacer predicciones en cada paso. También daba uina puntuacion de caracter de objeto, que explica la probabilidad estimada de que una imagen contenga urealmente un objeto entrado. Es una salida de clasificacion binaria utilizan sigmoide activacuion y entrenarse usando perdida entropia curzada binaria.Esta tecnica es batsante directa pero puede detectarse un objeto varuias veces.Alternativa:Usar redes completamente convolucionales.



- Redes completamente convolucionales:  Supongamos que la ultima capa de una red convolucional(de 100 mapas de caracteristicas, de 7x7 cada uno) recibe por último una capa densa con 200 neuronas, la capa de salida tendra un tensor de la forma(tamaño lote, 200(cada neurona de las 200 calcula suma ponderada de las 100x7x7 activaciones del kernel)). Ahora en vez de utilizar una capa densa vamos a usar una capa convolucional de 200 filtros y tamaño del kernel 7x7 con relleno valuid, generandose asi un tensor de forma (tamaño lote,1,1,200), es decir generara la misma cantidad de numero que la red densa pero ha cambiado la forma.

    Truco: Para utilizar la ultima capa convolucional como si fuera densa, el número de filtros debe ser igual al numero de neuronas a usar en la caoa densa, el tamaño del kernel de la convolucion debe ser igual al tamaño de los mapas de caracteristicas de entrada y hay que utilizar relleno "valid".


In [8]:
from tensorflow import keras

def crear_modelo_WDense():
    input_layer = keras.layers.Input((224,224,1))
    x = keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="valid",name="convolucional1")(input_layer)
    x = keras.layers.MaxPooling2D(strides=2,name="MaxPooling1")(x)
    x = keras.layers.Conv2D(filters=64,kernel_size=5,strides=2,padding="valid",name="convolucional2")(x)
    x = keras.layers.MaxPooling2D(strides=2,name="MaxPooling2")(x)
    x = keras.layers.Flatten(name="aplanamiento")(x)
    output_layer = keras.layers.Dense(30,name="salidadensa")(x)
    return keras.models.Model(inputs=input_layer,outputs=output_layer)

model = crear_modelo_WDense()
model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 224, 224, 1)]     0         
                                                                 
 convolucional1 (Conv2D)     (None, 111, 111, 32)      320       
                                                                 
 MaxPooling1 (MaxPooling2D)  (None, 55, 55, 32)        0         
                                                                 
 convolucional2 (Conv2D)     (None, 26, 26, 64)        51264     
                                                                 
 MaxPooling2 (MaxPooling2D)  (None, 13, 13, 64)        0         
                                                                 
 aplanamiento (Flatten)      (None, 10816)             0         
                                                                 
 salidadensa (Dense)         (None, 30)                3245

In [12]:
from tensorflow import keras

def crear_modelo_FConv():
    input_layer = keras.layers.Input((224,224,1))
    x = keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="valid",name="convolucional1")(input_layer)
    x = keras.layers.MaxPooling2D(strides=2,name="MaxPooling1")(x)
    x = keras.layers.Conv2D(filters=64,kernel_size=5,strides=2,padding="valid",name="convolucional2")(x)
    x = keras.layers.MaxPooling2D(strides=2,name="MaxPooling2")(x)
    output_layer = keras.layers.Conv2D(filters=30,kernel_size=13,padding="valid",name="salida")(x)
    return keras.models.Model(inputs=input_layer,outputs=output_layer)

model = crear_modelo_FConv()
model.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 224, 224, 1)]     0         
                                                                 
 convolucional1 (Conv2D)     (None, 111, 111, 32)      320       
                                                                 
 MaxPooling1 (MaxPooling2D)  (None, 55, 55, 32)        0         
                                                                 
 convolucional2 (Conv2D)     (None, 26, 26, 64)        51264     
                                                                 
 MaxPooling2 (MaxPooling2D)  (None, 13, 13, 64)        0         
                                                                 
 salida (Conv2D)             (None, 1, 1, 30)          324510    
                                                                 
Total params: 376094 (1.43 MB)
Trainable params: 376094 (1.